In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline

In [ ]:
def rk4(f, y0, h, start, stop):
    y = np.array([y0])
    t = np.array([start])
    for i in range(int(np.ceil((stop - start)/h))):
        k1 = h*f(y[-1], t[-1])
        k2 = h*f(y[-1] + k1/2, t[-1] + k1/2)
        k3 = h*f(y[-1] + k2/2, t[-1] + k2/2)
        k4 = h*f(y[-1] + k3, t[-1] + k3)
        y = np.concatenate((y, [y[-1] + (k1 + 2*k2 + 2*k3 + k4)/6]), 0)
        t = np.append(t, t[-1] + h)
    return (t, y)

In [ ]:
m = 1
l = 1
g = 9.81
def f(y, t):
    theta1 = y[0]
    theta2 = y[1]
    p1 = y[2]
    p2 = y[3]
    f1 = 6/(m*l**2)*(2*p1-3*np.cos(theta1-theta2)*p2)/(16-9*np.cos(theta1-theta2)**2)
    f2 = 6/(m*l**2)*(8*p2-3*np.cos(theta1-theta2)*p1)/(16-9*np.cos(theta1-theta2)**2)
    f3 = -0.5*m*l**2*(f1*f2*np.sin(theta1-theta2)+3*g/l*np.sin(theta1))
    f4 = -0.5*m*l**2*(-f1*f2*np.sin(theta1-theta2)+g/l*np.sin(theta2))
    return  np.array([f1, f2, f3, f4])

In [ ]:
data = rk4(f, [np.pi/2, np.pi/2, 0, 0], 0.001, 0, 20)
fact = 20

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.grid()
plt.gca().set_aspect('equal', adjustable='box')

line, = ax.plot([], [], 'r-', lw=3)
extremity, = ax.plot([], [], 'black', lw=1)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
x1 = l*np.sin(data[1][:,0])
y1 = -l*np.cos(data[1][:,0])
x2 = x1 + l*np.sin(data[1][:,1])
y2 = y1 - l*np.cos(data[1][:,1])

def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text

def animate(i):
    thisx = [0, x1[i*fact], x2[i*fact]]
    thisy = [0, y1[i*fact], y2[i*fact]]
    line.set_data(thisx, thisy)
    extremity.set_data(x2[:i*fact+1], y2[:i*fact+1])
    time_text.set_text(time_template % (data[0][i*fact]))
    return extremity, line, time_text

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=int(len(data[1])/fact), interval=0.001, blit=True)
anim.save("double_pendulum.mp4", fps=fact*5/2)

In [ ]:
%%HTML
<video width="600" height="600" controls>
   <source src="double_pendulum.mp4" type="video/mp4">
</video>